In [1]:
import os
import cv2
import numpy as np
import pandas as pd

In [7]:
images = []
labels = []
label = 0
name=[]

parent_folder = os.listdir("val_ten_uk_birds/")

for f in parent_folder:
    print(f)
    sub_folder=os.listdir("val_ten_uk_birds/"+f)
    name.append(f)
    for image in sub_folder:
        
        theImage=cv2.imread("val_ten_uk_birds/"+f+"/"+image)
        theImage=cv2.resize(theImage,(256,256))
        images.append(theImage)
        labels.append(label)
    label=label+1

04284_Animalia_Chordata_Aves_Passeriformes_Turdidae_Turdus_merula
04133_Animalia_Chordata_Aves_Passeriformes_Passeridae_Passer_domesticus
04034_Animalia_Chordata_Aves_Passeriformes_Paridae_Parus_major
03516_Animalia_Chordata_Aves_Columbiformes_Columbidae_Columba_palumbus
03978_Animalia_Chordata_Aves_Passeriformes_Muscicapidae_Erithacus_rubecula
03775_Animalia_Chordata_Aves_Passeriformes_Corvidae_Pica_pica
04031_Animalia_Chordata_Aves_Passeriformes_Paridae_Cyanistes_caeruleus
04202_Animalia_Chordata_Aves_Passeriformes_Sturnidae_Sturnus_vulgaris
03809_Animalia_Chordata_Aves_Passeriformes_Fringillidae_Carduelis_carduelis
03688_Animalia_Chordata_Aves_Passeriformes_Aegithalidae_Aegithalos_caudatus


In [8]:
data=[]
for i in range(len(images)):
    data.append([images[i], labels[i]])

In [9]:

data=np.array(data)
images=np.array(images)
labels=np.array(labels)
name=np.array(name)
np.save("np_arrays/images_and_labels_test.npy", data)
np.save("np_arrays/images.npy_test", images)
np.save("np_arrays/labels.npy_test", labels)
np.save("np_arrays/name_test.npy", name)

/tmp/ipykernel_12478/2775985548.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data=np.array(data)


In [10]:
pd_data=pd.DataFrame(data, columns=["Image" ,"Label"])

In [11]:
pd_data.head()

,Image,Label
0,"[[[218, 219, 209], [217, 218, 208], [217, 218,...",0
1,"[[[121, 138, 124], [122, 137, 123], [124, 136,...",0
2,"[[[65, 166, 115], [59, 158, 111], [64, 157, 11...",0
3,"[[[164, 136, 105], [191, 164, 134], [185, 157,...",0
4,"[[[70, 110, 91], [68, 110, 91], [70, 113, 92],...",0


In [12]:
pd_data.to_csv('np_arrays/pd_data_test.csv')

In [16]:
pd.value_counts(pd_data['Label'])

0    10
1    10
2    10
3    10
4    10
5    10
6    10
7    10
8    10
9    10
Name: Label, dtype: int64

In [29]:
i = []
for k in x['Image']:
    s = cv2.imread(np.array(k))
    print(s)
    i.append(s)
    j=1
    if j == 1:
        break

TypeError: Can't convert object of type 'numpy.ndarray' to 'str' for 'filename'

In [25]:
import matplotlib.pyplot as plt

In [26]:
print(i)

[None]
